In [1]:
import random
from tqdm.notebook import tqdm
from stox.alpha_vantage import AlphaVantageStocks
from stox.ibkr import load_symbol_table, save_symbol_table
import omnifig as fig
fig.initialize()

In [2]:
av = AlphaVantageStocks()
info = av.overview('BBVA')
info.keys()

dict_keys(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange', 'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd', 'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio', 'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS', 'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM', 'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM', 'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY', 'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE', 'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue', 'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage', '200DayMovingAverage', 'SharesOutstanding', 'DividendDate', 'ExDividendDate'])

In [3]:
symbols_table = load_symbol_table()
eur = {yfsym: info for yfsym, info in symbols_table.items() if info['ibkr-contract']['currency'] == 'EUR'}
len(symbols_table), len(eur)

(454, 239)

In [4]:
adr_srcs = {yfsym: info['adr'] for yfsym, info in symbols_table.items() if 'adr' in info}
adrs = {yfsym: info['adr-src'] for yfsym, info in symbols_table.items() if 'adr-src' in info}
assert all(adr_srcs[adrs[yfsym]] == yfsym for yfsym in adrs)
len(adr_srcs), len(adrs)

(182, 182)

In [5]:
nys = [adr for adr in adrs if symbols_table[adr]['ibkr-contract']['primaryExchange'] in {'NYSE', 'NASDAQ'}]
len(nys)

31

In [8]:
dl = fig.create_config(_type='downloader/alpha-vantage',).create_silent()
dl

In [9]:
dl.download_reports('BBVA', date='230909', pbar=tqdm)

  0%|          | 0/8 [00:00<?, ?it/s]

{}

In [21]:
date = '230909'
config = fig.create_config(**{'container._type': 'context', 
							  'container.gadgets.av._type': 'loader/alpha-vantage'})
_container_source = config.peek('container')
def create_container(yfsym):
	with _container_source.silence():
		ctx = _container_source.create()
	ctx['ticker'] = yfsym
	ctx['date'] = date
	return ctx

In [22]:
pop = {yfsym: create_container(yfsym) for yfsym in nys}
len(pop)

31

In [23]:
# ctx = pop[0]
# ctx = random.choice(pop)
ctx = pop['BBVA']
ctx

Context(ticker, date, {overview}, {income_statement}, {balance_sheet}, {cash_flow}, {earnings}, {earnings_calendar}, {ipo_calendar}, {listing_status})

In [24]:
overview = ctx['overview']
overview.keys()

dict_keys([])